<a href="https://colab.research.google.com/github/knc6/jarvis-tools-notebooks/blob/master/jarvis-tools-notebooks/Cubic_Crystal_Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook runs the jupyter-notebook given [here](https://github.com/materialsvirtuallab/m3gnet/blob/main/examples/Cubic%20Crystal%20Test.ipynb) on Colab.

In [5]:
!pip install m3gnet alignn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import traceback
import warnings

import numpy as np
import pandas as pd
from pymatgen.core import Composition, Lattice, Structure
#from pymatgen.ext.matproj import MPRester
#Run the cell twice to avoid MPRester error
#from pymatgen.ext.matproj import _MPResterLegacy as MPRester
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from tqdm import tqdm

from m3gnet.models import Relaxer

for category in (UserWarning, DeprecationWarning):
    warnings.filterwarnings("ignore", category=category, module="tensorflow")
    warnings.filterwarnings("ignore", category=category, module="pymatgen")

In [3]:
data = pd.read_html("http://en.wikipedia.org/wiki/Lattice_constant")[0]

In [4]:
data

,Material,Lattice constant (Å),Crystal structure,Ref.
0,C (diamond),3.567,Diamond (FCC),[7]
1,C (graphite),a = 2.461 c = 6.708,Hexagonal,NaN
2,Si,5.431020511,Diamond (FCC),[8][9]
3,Ge,5.658,Diamond (FCC),[8]
4,AlAs,5.6605,Zinc blende (FCC),[8]
...,...,...,...,...
86,CaVO3,3.767,Cubic perovskite,[15]
87,BaMnO3,a = 5.673 c = 4.71,Hexagonal,[15]
88,CaMnO3,a = 5.27 b = 5.275 c = 7.464,Orthorhombic perovskite,[15]
89,SrRuO3,a = 5.53 b = 5.57 c = 7.85,Orthorhombic perovskite,[15]


In [5]:

data = data[
    ~data["Crystal structure"].isin(
        ["Hexagonal", "Wurtzite", "Wurtzite (HCP)", "Orthorhombic", "Tetragonal perovskite", "Orthorhombic perovskite"]
    )
]
data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)

data["a (Å)"] = data["a (Å)"].map(float)


additional_fcc = """10 Ne 4.43 54 Xe 6.20
13 Al 4.05 58 Ce 5.16
18 Ar 5.26 70 Yb 5.49
20 Ca 5.58 77 Ir 3.84
28 Ni 3.52 78 Pt 3.92
29 Cu 3.61 79 Au 4.08
36 Kr 5.72 82 Pb 4.95
38 Sr 6.08 47 Ag 4.09
45 Rh 3.80 89 Ac 5.31
46 Pd 3.89 90 Th 5.08"""

additional_bcc = """3 Li 3.49 42 Mo 3.15
11 Na 4.23 55 Cs 6.05
19 K 5.23 56 Ba 5.02
23 V 3.02 63 Eu 4.61
24 Cr 2.88 73 Ta 3.31
26 Fe 2.87 74 W 3.16
37 Rb 5.59 41 Nb 3.30"""
def add_new(str_, structure_type, df):
    tokens = str_.split()
    new_crystals = []
    for i in range(int(len(tokens) / 3)):
        el = tokens[3 * i + 1].strip()
        if el not in df["Material"].values:
            new_crystals.append([tokens[3 * i + 1], structure_type, float(tokens[3 * i + 2])])
    df2 = pd.DataFrame(new_crystals, columns=data.columns)
    return pd.concat([df, df2])


# data = add_new(additional_fcc, "FCC", data)
# data = add_new(additional_bcc, "BCC", data)
# data.set_index("Material", inplace=True)
print(data)

       Material     a (Å)  Crystal structure    Ref.
0   C (diamond)  3.567000      Diamond (FCC)     [7]
2            Si  5.431021      Diamond (FCC)  [8][9]
3            Ge  5.658000      Diamond (FCC)     [8]
4          AlAs  5.660500  Zinc blende (FCC)     [8]
5           AlP  5.451000  Zinc blende (FCC)     [8]
..          ...       ...                ...     ...
79        KTaO3  3.988500   Cubic perovskite    [15]
81       SrTiO3  3.988050   Cubic perovskite    [15]
84       EuTiO3  7.810000   Cubic perovskite    [15]
85        SrVO3  3.838000   Cubic perovskite    [15]
86        CaVO3  3.767000   Cubic perovskite    [15]

[73 rows x 4 columns]


<ipython-input-5-08c8062c8bd3>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={"Lattice constant (Å)": "a (Å)"}, inplace=True)
<ipython-input-5-08c8062c8bd3>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["a (Å)"] = data["a (Å)"].map(float)


Setting 5 Angstrom lattice constant to all crystals

In [6]:
MAPI_KEY_LEGACY = '' #Optional, keep your MAPI_LEGACY_KEY here
predicted = []
mp = []
initial_structs = []
final_m3gnet_structs = []

#mpr = MPRester(MAPI_KEY_LEGACY)

relaxer = Relaxer()  # This loads the default pre-trained model

# warnings.filterwarnings(action="ignore", category=UserWarning, module="tensorflow")
currently_m3gnet_failing = ['CsCl','CsI','Fe','Mo','W','V','Nb','Ta','Ne','CaVO3','Xe','Ar','Yb','Kr','Sr','Ac','Li','Na','Cs','K','Ba','Eu','Cr','Rb',"NC0.99"]

#for formula, v in tqdm(data.iterrows(), total=len(data)):
for ii,i in tqdm(data.iterrows(), total=len(data)):
  #M3gnet failing so skipping these
  formula=i['Material']
  if formula not in currently_m3gnet_failing: 
    formula = formula.split()[0]
    c = Composition(formula)
    els = sorted(c.elements)
    #cs = v["Crystal structure"]
    cs = i["Crystal structure"]
    wiki = i['a (Å)']
    # We initialize all the crystals with an arbitrary lattice constant of 5 angstroms.
    if "Zinc blende" in cs:
        s = Structure.from_spacegroup("F-43m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.25, 0.25, 0.75]])
    elif "Halite" in cs:
        s = Structure.from_spacegroup("Fm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0, 0]])
    elif "Caesium chloride" in cs:
        s = Structure.from_spacegroup("Pm-3m", Lattice.cubic(5), [els[0], els[1]], [[0, 0, 0], [0.5, 0.5, 0.5]])
    elif "Cubic perovskite" in cs:
        s = Structure(
            Lattice.cubic(5),
            [els[0], els[1], els[2], els[2], els[2]],
            [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]],
        )
    elif "Diamond" in cs:
        s = Structure.from_spacegroup("Fd-3m", Lattice.cubic(5), [els[0]], [[0.25, 0.75, 0.25]])
    elif "BCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 2, [[0.0, 0.0, 0.0], [0.5, 0.5, 0.5]])
    elif "FCC" in cs:
        s = Structure(Lattice.cubic(5), [els[0]] * 4, [[0.0, 0.0, 0.0], [0.5, 0.5, 0], [0.0, 0.5, 0.5], [0.5, 0, 0.5]])
    else:
        predicted.append(0)
        mp.append(0)
        continue
    print(s)
    initial_structs.append(s)
    relax_results = relaxer.relax(s)

    final_structure = relax_results["final_structure"]
    final_m3gnet_structs.append(final_structure)
    predicted.append(final_structure.lattice.a)
    
    #break
    try:
        mids = mpr.get_entries(s.composition.reduced_formula)
        for i in mids:
            try:
                structure = mpr.get_structure_by_material_id(i.entry_id)
                sga = SpacegroupAnalyzer(structure)
                sga2 = SpacegroupAnalyzer(final_structure)
                if sga.get_space_group_number() == sga2.get_space_group_number():
                    conv = sga.get_conventional_standard_structure()
                    mp.append(conv.lattice.a)
                    break
            except Exception:
                pass
        else:
            raise RuntimeError
    except Exception:
        mp.append(0)
        traceback.print_exc()
    print('\nInitial,final,MP,Wiki,formula',5,final_structure.lattice.a,mp[-1],wiki)

  0%|          | 0/73 [00:00<?, ?it/s]

Full Formula (C8)
Reduced Formula: C
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  C     0.25  0.25  0.75
  1  C     0     0.5   0
  2  C     0.75  0.75  0.75
  3  C     0.5   0.5   0.5
  4  C     0.75  0.25  0.25
  5  C     0.5   0     0
  6  C     0     0     0.5
  7  C     0.25  0.75  0.25


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
  1%|▏         | 1/73 [00:14<17:47, 14.82s/it]


Initial,final,MP,Wiki,formula 5 3.5718039756536863 0 3.567
Full Formula (Si8)
Reduced Formula: Si
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (8)
  #  SP       a     b     c
---  ----  ----  ----  ----
  0  Si    0.25  0.25  0.75
  1  Si    0     0.5   0
  2  Si    0.75  0.75  0.75
  3  Si    0.5   0.5   0.5
  4  Si    0.75  0.25  0.25
  5  Si    0.5   0     0
  6  Si    0     0     0.5
  7  Si    0.25  0.75  0.25


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
  3%|▎         | 2/73 [00:15<07:29,  6.33s/it]

...


Initial,final,MP,Wiki,formula 5 3.9448066067159178 0 3.98805
Full Formula (Eu1 Ti1 O3)
Reduced Formula: EuTiO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Eu    0    0    0
  1  Ti    0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
 97%|█████████▋| 71/73 [01:06<00:01,  1.23it/s]


Initial,final,MP,Wiki,formula 5 3.9294272293452503 0 7.81
Full Formula (Sr1 V1 O3)
Reduced Formula: SrVO3
abc   :   5.000000   5.000000   5.000000
angles:  90.000000  90.000000  90.000000
pbc   :       True       True       True
Sites (5)
  #  SP      a    b    c
---  ----  ---  ---  ---
  0  Sr    0    0    0
  1  V     0.5  0.5  0.5
  2  O     0.5  0.5  0
  3  O     0    0.5  0.5
  4  O     0.5  0    0.5


Traceback (most recent call last):
  File "<ipython-input-6-a9bc7b7bd966>", line 58, in <cell line: 15>
    mids = mpr.get_entries(s.composition.reduced_formula)
NameError: name 'mpr' is not defined
100%|██████████| 73/73 [01:07<00:00,  1.07it/s]


Initial,final,MP,Wiki,formula 5 3.906043550979565 0 3.838


In [7]:
data_1 = data[~data["Material"].isin(currently_m3gnet_failing)]

In [8]:
data_1

,Material,a (Å),Crystal structure,Ref.
0,C (diamond),3.567000,Diamond (FCC),[7]
2,Si,5.431021,Diamond (FCC),[8][9]
3,Ge,5.658000,Diamond (FCC),[8]
4,AlAs,5.660500,Zinc blende (FCC),[8]
5,AlP,5.451000,Zinc blende (FCC),[8]
...,...,...,...,...
77,ScN,4.520000,Halite,[14]
79,KTaO3,3.988500,Cubic perovskite,[15]
81,SrTiO3,3.988050,Cubic perovskite,[15]
84,EuTiO3,7.810000,Cubic perovskite,[15]


In [9]:
data_1["MP a (Å)"] = mp
data_1["Predicted a (Å)"] = predicted
data_1["initial_structs"] = initial_structs
data_1["final_m3gnet_structs"] = final_m3gnet_structs

<ipython-input-9-02a3f8920e94>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["MP a (Å)"] = mp
<ipython-input-9-02a3f8920e94>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_1["Predicted a (Å)"] = predicted
<ipython-input-9-02a3f8920e94>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

In [10]:
from sklearn.metrics import mean_absolute_error
print('MAE wrt Exp/Wiki data',mean_absolute_error(data_1['a (Å)'],data_1['Predicted a (Å)']))
print('MAE wrt MP data',mean_absolute_error(data_1['MP a (Å)'],data_1['Predicted a (Å)']))

MAE wrt Exp/Wiki data 0.13163907193121901
MAE wrt MP data 5.175435282714427


In [26]:
from jarvis.core.atoms import Atoms
from alignn.ff.ff import ForceField, default_path
from jarvis.core.atoms import pmg_to_atoms
model_path = default_path()
def optimize_atoms(atoms=[],model_path=model_path):
    atoms = pmg_to_atoms(atoms)
    print(atoms.composition.reduced_formula)
    ff = ForceField(
        jarvis_atoms=atoms,
        model_path=model_path,
        stress_wt=.5,
        force_mult_natoms=False
    )
    opt, en, fs = ff.optimize_atoms()
    return opt.lattice.a

model_path /usr/local/lib/python3.10/dist-packages/alignn/ff


In [ ]:
tqdm.pandas()
#slow
data_1['alignn_a']=data_1['initial_structs'].progress_apply(lambda x:optimize_atoms(atoms=x))

  0%|          | 0/63 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/dgl/backend/pytorch/tensor.py:445: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  assert input.numel() == input.storage().size(), (


C
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-17.14334 eV KE= 0.00000 eV T= 0.000 K 
a= 4.939 Ang b= 4.939 Ang c= 4.939 Ang Volume= 120.491 amu/a3 PE=-17.95782 eV KE= 0.00000 eV T= 0.000 K 
a= 4.817 Ang b= 4.817 Ang c= 4.817 Ang Volume= 111.751 amu/a3 PE=-19.64065 eV KE= 0.00000 eV T= 0.000 K 
a= 4.627 Ang b= 4.623 Ang c= 4.623 Ang Volume= 98.891 amu/a3 PE=-23.75571 eV KE= 0.00000 eV T= 0.000 K 
a= 4.334 Ang b= 4.316 Ang c= 4.316 Ang Volume= 80.706 amu/a3 PE=-32.83384 eV KE= 0.00000 eV T= 0.000 K 
a= 3.987 Ang b= 3.943 Ang c= 3.943 Ang Volume= 61.974 amu/a3 PE=-46.16742 eV KE= 0.00000 eV T= 0.000 K 
a= 3.579 Ang b= 3.513 Ang c= 3.513 Ang Volume= 44.093 amu/a3 PE=-61.27943 eV KE= 0.00000 eV T= 0.000 K 
a= 3.579 Ang b= 3.513 Ang c= 3.513 Ang Volume= 44.105 amu/a3 PE=-61.27988 eV KE= 0.00000 eV T= 0.000 K 
a= 3.580 Ang b= 3.514 Ang c= 3.514 Ang Volume= 44.129 amu/a3 PE=-61.28079 eV KE= 0.00000 eV T= 0.000 K 
a= 3.580 Ang b= 3.515 Ang c= 3.515 Ang Volume=

  3%|▎         | 2/63 [00:56<28:48, 28.34s/it]

Si
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-24.25818 eV KE= 0.00000 eV T= 0.000 K 
a= 5.612 Ang b= 5.612 Ang c= 5.612 Ang Volume= 176.748 amu/a3 PE=-32.17567 eV KE= 0.00000 eV T= 0.000 K 
a= 5.605 Ang b= 5.605 Ang c= 5.605 Ang Volume= 176.118 amu/a3 PE=-32.20227 eV KE= 0.00000 eV T= 0.000 K 
a= 5.592 Ang b= 5.592 Ang c= 5.592 Ang Volume= 174.878 amu/a3 PE=-32.25393 eV KE= 0.00000 eV T= 0.000 K 
a= 5.573 Ang b= 5.572 Ang c= 5.573 Ang Volume= 173.065 amu/a3 PE=-32.32642 eV KE= 0.00000 eV T= 0.000 K 
a= 5.547 Ang b= 5.548 Ang c= 5.548 Ang Volume= 170.732 amu/a3 PE=-32.41366 eV KE= 0.00000 eV T= 0.000 K 
a= 5.516 Ang b= 5.517 Ang c= 5.518 Ang Volume= 167.946 amu/a3 PE=-32.50969 eV KE= 0.00000 eV T= 0.000 K 
a= 5.481 Ang b= 5.483 Ang c= 5.483 Ang Volume= 164.782 amu/a3 PE=-32.60809 eV KE= 0.00000 eV T= 0.000 K 
a= 5.444 Ang b= 5.444 Ang c= 5.443 Ang Volume= 161.317 amu/a3 PE=-32.70315 eV KE= 0.00000 eV T= 0.000 K 
a= 5.398 Ang b= 5.397 Ang c= 5.396 Ang 

  5%|▍         | 3/63 [02:21<52:00, 52.00s/it]

...

AlAs
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-34.00023 eV KE= 0.00000 eV T= 0.000 K 
a= 5.154 Ang b= 5.154 Ang c= 5.154 Ang Volume= 136.899 amu/a3 PE=-11.61459 eV KE= 0.00000 eV T= 0.000 K 
a= 5.605 Ang b= 5.611 Ang c= 5.613 Ang Volume= 176.535 amu/a3 PE=-25.51973 eV KE= 0.00000 eV T= 0.000 K 
a= 6.136 Ang b= 6.146 Ang c= 6.153 Ang Volume= 232.028 amu/a3 PE=-25.24877 eV KE= 0.00000 eV T= 0.000 K 
a= 6.129 Ang b= 6.139 Ang c= 6.147 Ang Volume= 231.261 amu/a3 PE=-25.27199 eV KE= 0.00000 eV T= 0.000 K 
a= 6.116 Ang b= 6.125 Ang c= 6.133 Ang Volume= 229.746 amu/a3 PE=-25.31719 eV KE= 0.00000 eV T= 0.000 K 
a= 6.096 Ang b= 6.105 Ang c= 6.113 Ang Volume= 227.516 amu/a3 PE=-25.38193 eV KE= 0.00000 eV T= 0.000 K 
a= 6.070 Ang b= 6.079 Ang c= 6.088 Ang Volume= 224.623 amu/a3 PE=-25.46239 eV KE= 0.00000 eV T= 0.000 K 
a= 6.039 Ang b= 6.047 Ang c= 6.056 Ang Volume= 221.136 amu/a3 PE=-25.55351 eV KE= 0.00000 eV T= 0.000 K 
a= 6.002 Ang b= 6.010 Ang c= 6.019 An

  8%|▊         | 5/63 [03:40<43:35, 45.09s/it]

AlP
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE=-21.95017 eV KE= 0.00000 eV T= 0.000 K 
a= 5.232 Ang b= 5.232 Ang c= 5.232 Ang Volume= 143.235 amu/a3 PE=-29.14803 eV KE= 0.00000 eV T= 0.000 K 
a= 5.561 Ang b= 5.564 Ang c= 5.561 Ang Volume= 172.085 amu/a3 PE=-30.75252 eV KE= 0.00000 eV T= 0.000 K 


 10%|▉         | 6/63 [03:55<33:14, 34.99s/it]

AlSb
OPTIMIZATION
a= 5.000 Ang b= 5.000 Ang c= 5.000 Ang Volume= 125.000 amu/a3 PE= 5.65614 eV KE= 0.00000 eV T= 0.000 K 
a= 5.398 Ang b= 5.398 Ang c= 5.398 Ang Volume= 157.276 amu/a3 PE= 5.15569 eV KE= 0.00000 eV T= 0.000 K 
a= 6.058 Ang b= 6.062 Ang c= 6.055 Ang Volume= 222.382 amu/a3 PE=-19.32637 eV KE= 0.00000 eV T= 0.000 K 
a= 6.794 Ang b= 6.809 Ang c= 6.797 Ang Volume= 314.434 amu/a3 PE=-19.32217 eV KE= 0.00000 eV T= 0.000 K 
a= 6.788 Ang b= 6.803 Ang c= 6.791 Ang Volume= 313.619 amu/a3 PE=-19.33641 eV KE= 0.00000 eV T= 0.000 K 
a= 6.777 Ang b= 6.791 Ang c= 6.780 Ang Volume= 312.010 amu/a3 PE=-19.36390 eV KE= 0.00000 eV T= 0.000 K 
a= 6.760 Ang b= 6.774 Ang c= 6.763 Ang Volume= 309.648 amu/a3 PE=-19.40267 eV KE= 0.00000 eV T= 0.000 K 
a= 6.738 Ang b= 6.751 Ang c= 6.741 Ang Volume= 306.592 amu/a3 PE=-19.44987 eV KE= 0.00000 eV T= 0.000 K 
a= 6.711 Ang b= 6.723 Ang c= 6.714 Ang Volume= 302.921 amu/a3 PE=-19.50191 eV KE= 0.00000 eV T= 0.000 K 
a= 6.681 Ang b= 6.690 Ang c= 6.683 Ang 

In [ ]:
pip freeze